In [ ]:
import ujson
from dutchanalyzer.config import *
from dutchanalyzer.utilities.utils import *
from dutchanalyzer.utilities.json_utils import *
from dutchanalyzer.utilities.replacement_utils import *
from pathlib import Path
from io import StringIO
import datetime
import re
from dotenv import load_dotenv
from pprint import pprint
import ast

### Paths

In [2]:
ERAW_FILE = Path(RAW_KAIKKI_DIR, 'en', 'kaikki_en-raw-wiktextract-data.jsonl') 
NRAW_FILE = Path(RAW_KAIKKI_DIR, 'nl', 'kaikki_nl-raw-extract.jsonl')

In [3]:
eng_save_path = Path(WIKT_PREPROCESSING_DIR, 'en')
nld_save_path = Path(WIKT_PREPROCESSING_DIR, 'nl')

### Shared Utilities

## Process Raw NL File

### Intake JSON

In [4]:
# Paths
NL_lines_file = Path(nld_save_path, 'NLR.jsonl')
EN_lines_file = Path(eng_save_path, 'ELR.jsonl')
ENR_file = Path(ENR_DIR, 'ENR.jsonl')
EER_file = Path(EER_DIR, 'EER.jsonl')
NNR_file = Path(NNR_DIR, 'NNR.jsonl')
NER_file = Path(NER_DIR, 'NER.jsonl')
ENR_file2 = Path(ENR_DIR, 'ENR2.jsonl')
EER_file2 = Path(EER_DIR, 'EER2.jsonl')
NNR_file2 = Path(NNR_DIR, 'NNR2.jsonl')
NER_file2 = Path(NER_DIR, 'NER2.jsonl')
other_file = Path(WIKT_PREPROCESSING_DIR, 'other_langs.jsonl')

In [ ]:
def filter_obj(obj):
    lang_codes_to_keep = ['nl', 'en', 'simple', 'ang', 'dum', 'nds', 'odt', 'nds-nl', 'enm', 'eng', 'nld']
    switch_to_english_langs = {'nederlands': 'dutch', 'engels': 'english', 'oudengels': 'old_english', 'angelsaksisch': 'anglo_saxon', 'oudengels': 'old_english', 'oud engels':'old_english', 'old english':'old_english', 'oldenglish': 'old_english','middelnederlands':'middle_dutch', 'middle english': 'middle_english', 'middle dutch': 'middle_dutch', 'middelengels': 'middle_english', 'simple':'simple_english'}

    langs_to_keep = ['english', 'engels', 'dutch', 'nederlands', 'middelnederlands', 'middle dutch', 'simple', 'middle english', 'angelsaksisch', 'old english', 'oudengels', 'oud engels', 'oud engels', 'middelengels', 'middel engels', 'oudengels']
    new_sounds = []
    new_derived =[]
    new_decendents = []
    new_translations = []
    new_categories = []
    new_senses = []
    if 'anagrams' in obj:
        obj.pop('anagrams')
    if 'proverbs' in obj:
        obj.pop('proverbs')
    if 'pos_title' in obj:
        obj.pop('pos_title')
    
    if 'sounds' in obj:
        sounds = obj['sounds']
        remove_sounds = ['audio','ogg_url','mp3_url']
        new_sounds = []
        for i in sounds:
            ns = []
            for k in i.keys():
                if k not in remove_sounds:
                    ns.append(i)
            if ns:
                new_sounds.append(ns)
        obj['sounds'] = new_sounds
    if 'descendants' in obj:
        descendants = obj['descendants']
        new_decendents = []
        for d in descendants:
            dlang_code = d.get('lang_code', '')
            dlang = d.get('lang', '')
            if lookup_lang_from_code(dlang_code) or lookup_lang_code(dlang):
                new_decendents.append(d)
        obj['descendants'] = new_decendents
     
    if 'senses' in obj:
        new_senses = []
        for i, sense in enumerate(obj['senses']):
            if 'wikidata' in sense:
                sense.pop('wikidata')
            if 'wikipedia' in sense:
                sense.pop('wikipedia')
            if 'senseid' in sense:
                sense.pop('senseid')
            if 'translations' in sense:
                sense_translations = [s for s in sense['translations'] if s.get('lang_code') in lang_codes_to_keep]
                sense['translations'] = sense_translations
            if 'translation' in sense and 'translations' not in sense:
                sense_translations = [s for s in sense['translation'] if s.get('lang_code') in lang_codes_to_keep]
                sense['translation'] = sense_translations
            new_senses.append(sense)
        obj['senses'] = new_senses
    if "categories" in obj:
        categories = obj["categories"]
        new_categories = []
        if isinstance(categories, list):
            for i, cat in enumerate(categories):
                if isinstance(cat, str):
                    if cat not in ["Terms with Dutch translations", "Terms with English translations"]:
                        if cat.startswith('Terms with') and cat.endswith('translations'):
                            categories.remove(cat)
                        else:
                            new_categories.append(cat)
        obj["categories"] = new_categories
    if 'translations' in obj:
        translations = obj['translations']
        for t in new_translations:
            tlang_code = t.get('lang_code', '').lower()
            tlang = t.get('lang', '').lower()
            if tlang_code in lang_codes_to_keep:
                new_translations.append(t)
            elif tlang in langs_to_keep: 
                translations = [t for t in translations if t.get("lang_code") in lang_codes_to_keep]
                obj['translations'] = translations


In [18]:
def filter_en_translations_regex(obj_str: str):
    translations_pattern = r'"translations"\s*:\s*\[({.*?})\]'
    en_translation_pattern = r'\{[^{}]*?"lang"\s*:\s*"Engels"[^{}]*?\}'
    translations_block = re.compile(translations_pattern, re.DOTALL)
    en_object = re.compile(en_translation_pattern, re.DOTALL)
    match = translations_block.search(obj_str)
    
    # while match is not None:
    if match:
        start, end = match.span()
        m = match.group(0)
        lang_code_i = m.find('"lang_code":')
        lang_i = m.find('"lang":')
        if lang_code_i != -1:
            print(m[lang_code_i: lang_code_i + 15])
        elif lang_i != -1:
            print(m[lang_i: lang_i + 20])

        # if dn:
        #     dn = [safe_dict(x) for x in dn]
        #     str_dn = '[' + ', '.join(json.dumps(x) for x in dn) + ']'
        #     obj_str = obj_str[:start] + '"translations": ' + str_dn + obj_str[end:]
        #     match = translations_block.search(obj_str, start + len('"translations": ' + str(dn)))
        # else:
            
        #     to_remove_end = end
        #     if end < len(obj_str) and obj_str[end] == ',':
        #         to_remove_end += 1
        #     obj_str = obj_str[:start] + obj_str[to_remove_end:]
        #     match = translations_block.search(obj_str, start)

    return obj_str

In [26]:
def filter_translations_regex(obj_str: str):
    translations_pattern = r'"translations"\s*:\s*\[({.*?})\]'
    translations_block = re.compile(translations_pattern, re.DOTALL)
    gen_lang_block_pattern = r'\{[^{}]*?"lang"\s*:\s*"(English|Engels|Dutch|Nederlands|Old English|Oudengels|Old Saxon|Oudnederlands|Dutch Low Saxon|Middle Dutch|Middelnederlands|Old Dutch|Middle English|Limburgish|Oudnederlands|Middenengels|Middelengels|Simple English|Eenvoudig Engels)"[^{}]*?\}'
    rarer_lang_block_patterns = r'\{[^{}]*?"lang"\s*:\s*"(Old English|Oudengels|Old Saxon|OudenNederlands|Dutch Low Saxon|Middle Dutch|Old Dutch|Middle English|Limburgish)"[^{}]*?\}'
    gen_lang_block = re.compile(gen_lang_block_pattern, re.DOTALL)
    en_translation_pattern = r'\{[^{}]*?"lang_code"\s*:\s*"(en|eng)"[^{}]*?\}'
    nl_translation_pattern = r'\{[^{}]*?"lang_code"\s*:\s*"nl"[^{}]*?\}'
    engels_translation_pattern = r'\{[^{}]*?"lang"\s*:\s*"Engels"[^{}]*?\}'
    english_translation_pattern = r'\{[^{}]*?"lang"\s*:\s*"English"[^{}]*?\}'
    dutch_translation_pattern = r'\{[^{}]*?"lang"\s*:\s*"Dutch"[^{}]*?\}'
    nederlands_translation_pattern = r'\{[^{}]*?"lang"\s*:\s*"Nederlands"[^{}]*?\}'
    match = translations_block.search(obj_str)
    
    while match is not None:
        if match:
            new_translations_str = '"translations": ['
            start, end = match.span()
            m = match.group(0)
            dn = gen_lang_block.findall(m)

        if dn:
            dn = [safe_dict(x) for x in dn]
            str_dn = '[' + ', '.join(json.dumps(x) for x in dn) + ']'
            obj_str = obj_str[:start] + '"translations": ' + str_dn + obj_str[end:]
            match = translations_block.search(obj_str, start + len('"translations": ' + str(dn)))
        else:
            to_remove_end = end
            if end < len(obj_str) and obj_str[end] == ',':
                to_remove_end += 1
            obj_str = obj_str[:start] + obj_str[to_remove_end:]
            match = translations_block.search(obj_str, start)

    return obj_str

In [50]:
def remove_obj_keys(obj:dict, keys=['anagrams', 'proverbs', 'pos_title']):
    for k in keys:
        if k in obj:
            obj.pop(k, None)   
    return obj

In [ ]:
def fast_sort_obj(obj, by='keys', start_keys=['word', 'pos', 'lang_code'], end_keys=['translations'], list_subkey_sort='lang_code'):
    """ 
    takes in list, list of dicts, or dict. 
    If keys, will sort keys leaving set started keys. 
    Else if a list of strings will sort strings. If a list of dicts will sort. 
    If subkey sort then list of keys will also be sorted by that subkey. 
    """
    if not obj:
        return None
    if not isinstance(obj, [dict, list]):
        print('Not list or dict')
        raise
    elif isinstance(obj, dict):
        new_start_keys = []
        new_end_keys = []
        if by == 'keys':
           items = sorted(obj.items()) 

In [ ]:
def nl_keep_before_load(line: str) -> bool:
    if has_cjk_or_arabic_fast(line, 30):
        return False
    return True
    

In [53]:
def lookup_lang_from_code(lang_code):
    if not lang_code:
        return None
    codes_to_names = {
    "en": {
        "standard": "english",
        "en_name": "English",
        "nl_name": "Engels"
    },
    "eng": {
        "standard": "english",
        "en_name": "English",
        "nl_name": "Engels"
    },
    "nl": {
        "standard": "dutch",
        "en_name": "Dutch",
        "nl_name": "Nederlands"
    },
    "nld": {
        "standard": "dutch",
        "en_name": "Dutch",
        "nl_name": "Nederlands"
    },
    "simple": {
        "standard": "simple_english",
        "en_name": "Simple English",
        "nl_name": "Eenvoudig Engels"
    },
    "ang": {
        "standard": "old_english",
        "en_name": "Old English",
        "nl_name": "Oudengels"
    },
    "enm": {
        "standard": "middle_english",
        "en_name": "Middle English",
        "nl_name": "Middelengels"
    },
    "dum": {
        "standard": "middle_dutch",
        "en_name": "Middle Dutch",
        "nl_name": "Middelnederlands"
    },
    "odt": {
        "standard": "old_dutch",
        "en_name": "Old Dutch",
        "nl_name": "Oudnederlands"
    },
    "nds": {
        "standard": "low_saxon",
        "en_name": "Low German / Low Saxon",
        "nl_name": "Nedersaksisch"
    },
    "nds-nl": {
        "standard": "dutch_low_saxon",
        "en_name": "Dutch Low Saxon",
        "nl_name": "Nedersaksisch (Nederland)"
    },
    'lim': {
        'standard': 'limburgish'
    },
    'osx': {
        'standard': 'pre_dutch'
        }
    }
    if lang_code:
        name = codes_to_names.get(lang_code)
        if name:
            return name['standard']
        else:
            return None
    return None



In [ ]:
#
def standardize_translation(obj: dict, lang_codes_to_keep=[], keep_no_lang=False, source='EEF', sense_index=-1) -> dict | None:

    if lang_codes_to_keep == []:
        lang_codes_to_keep = ['nl', 'en', 'simple', 'ang', 'dum', 'nds', 'odt', 'nds-nl', 'enm', 'eng', 'nld']

    word = obj.get('word', '')
    sense = obj.get('sense', '')

    if word == '' and sense == '':
        return None
    
    lang_code = obj.get('lang_code', '').lower()
    lang = obj.get('lang', '').lower()
    
    if lang_code == '':
        lang_code = obj.get('code', '').lower()
        
    new_translation = {}
    if keep_no_lang == False and lang_code == '' and lang == '':
        return None

    if lang_code == '' and lang == '':
        lang_code = 'unk'
        lang = 'unknown'

    elif lang == '':
        lang = lookup_lang_from_code(lang_code)

    elif lang_code == '':
        lang_code = lookup_lang_code(lang)

        if lang_code not in lang_codes_to_keep:
            return None
    
    standard_lang = lookup_lang_from_code(lang_code)
    if not standard_lang:
        return None
    pos = obj.get('pos')
    new_translation = {'word': word,
                        'lang_code': lang_code,
                        'lang': lang,
                        'standard_lang': standard_lang}
    
    obj_items = sorted(obj.items())
    for key, val in obj_items:
        if key not in new_translation:
            new_translation[key] = val
    return new_translation

In [ ]:
# Moved
def sort_translations(translations: list, sort_by='lang_code') -> list:
    if not translations:
        return []
    sorted_translations = sorted(translations, key=lambda translations: translations[sort_by])
    return sorted_translations

In [57]:
def sort_entry(obj, code=None, lang=None, standard_lang=None):
    
    new_obj = OrderedDict()
    new_obj['word'] = obj.get('word')
    new_obj['pos'] = obj.get('pos', 'unk')
    if code:
        new_obj['lang_code'] = code
    else:
        code = obj.get('lang_code')
        new_obj['lang_code'] = code
    if lang:
        new_obj['lang'] = lang
    else: new_obj['lang'] = obj.get('lang')

    if standard_lang:
        new_obj['standard_lang'] = standard_lang
    else:
        standard_lang = obj.get('standard_lang')
        if not standard_lang:
            standard_lang = lookup_lang_from_code(code)
        if not standard_lang:
            return None
        new_obj['standard_lang'] = standard_lang

    new_obj['senses'] = obj.get('senses')
    new_translations = []
    translations = obj.get('translations')
    
    if translations:
        for t in translations:
            translation = standardize_translation(t)
            if translation:
                new_translations.append(translation)
        new_translations = sort_translations(new_translations)
    else:
        new_translations = []
    t = obj.pop('translations', None)
    sorted_items = sorted(obj.items())
    for k, v in sorted_items:
        if k not in new_obj.keys():
            new_obj[k] = v
    new_obj['translations'] = new_translations
    return new_obj

In [62]:
def filter_nl_obj(obj: dict):
    
    word = obj.get('word')
    if not word:
        return None
    if has_cjk_or_arabic_fast(word):
        return None
    if not check_has_valid_chars(word):
        return None
    
    obj = remove_obj_keys(obj)
    if 'sounds' in obj:
        sounds = obj['sounds']
        remove_sounds = ['audio','ogg_url','mp3_url']
        new_sounds = []
        for i in sounds:
            ns = []
            for k in i.keys():
                if k not in remove_sounds:
                    ns.append(i)
            if ns:
                new_sounds.append(ns)
        obj['sounds'] = new_sounds  
    new_obj = sort_entry(obj)            
    return new_obj

    

In [ ]:
error_lines = []
batch_size = 10000
batch = []
NER_lines = []
other_lines = []
file_path = NRAW_FILE
out_file = NNR_file2
with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
    with open(out_file, 'w+', encoding='utf-8', errors='ignore') as out:
        lines = f.readlines()

        for i, line in tqdm(enumerate(lines), total=len(lines)):
            if nl_keep_before_load(line):
                try:
                    loaded = json.loads(line)
                    if loaded:
                        obj = filter_nl_obj(loaded)
                        if obj:
                            if obj['lang_code'] == 'nl':
                                batch.append(obj)
                                if len(batch) >= batch_size:
                                    for obj in batch:
                                        json.dump(obj, out, ensure_ascii=False)
                                        out.write('\n')
                                        batch = [] 
                            elif obj['lang_code'] == 'en':
                                NER_lines.append(obj)
                            else:
                                other_lines.append(obj)
                except Exception as e:
                    print(f"Error on line: {i}")
                    print(e)
                    break
        if batch:
            for obj in batch:
                json.dump(obj, out, ensure_ascii=False)
                out.write('\n')
                batch = [] 

  1%|          | 8381/1050145 [00:00<00:59, 17530.39it/s]

{'title': 'Hoofdpagina', 'redirect': 'WikiWoordenboek:Hoofdpagina', 'pos': 'hard-redirect'}


100%|██████████| 1050145/1050145 [03:10<00:00, 5502.41it/s] 


In [64]:
out_file = NER_file2
with open(out_file, 'w+', encoding='utf-8', errors='ignore') as out:
    for obj in NER_lines:
        json.dump(obj, out, ensure_ascii=False)
        out.write('\n')
with open(other_file, 'a+', encoding='utf-8', errors='ignore') as out:
    for obj in other_lines:
        json.dump(obj, out, ensure_ascii=False)
        out.write('\n')

In [47]:
# The nl file is small enough to intake by readlinesbatch = []
error_lines = []
batch_size = 10000
batch = []
file_path = NRAW_FILE
out_file = NL_lines_file
with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
    with open(out_file, 'w+', encoding='utf-8', errors='ignore') as out:
        lines = f.readlines()

        for i, line in tqdm(enumerate(lines), total=len(lines)):
            
            if nl_keep_before_load(line):
                try:
                    loaded = json.loads(line)
                    if loaded:
                        
                            
                        obj = filter_obj(loaded)
                        if obj:
                            batch.append(obj)
                            if len(batch) >= batch_size:
                                for obj in batch:
                                    json.dump(obj, out, ensure_ascii=False)
                                    out.write('\n')
                                    batch = [] 
                except Exception as e:
                    print(f"Error on line: {i}")
                    print(e)
                    break
        if batch:
            for obj in batch:
                json.dump(obj, out, ensure_ascii=False)
                out.write('\n')
                batch = [] 

100%|██████████| 1050145/1050145 [01:10<00:00, 14912.22it/s]


In [ ]:
other = []
ner_lines = []
nnr_lines = []
with open(NL_lines_file, 'r', encoding='utf-8') as f:
    with open(NNR_lines_file, 'w+', encoding='utf-8') as out:
        lines = f.readlines()
        print(len(lines))
        for line in tqdm(lines, desc='Sorting lines'):
            loaded = json.loads(line)
            code = loaded.get('lang_code')
            if not code:
                loaded.get('code')
            
            if code == 'nl' or loaded.get('lang', '') == 'Nederlands':
                nnr_lines.append(loaded)
            elif code == 'en' or loaded.get('lang', '') == 'Engels':
                ner_lines.append(loaded)
            else:
                other.append(loaded)
        if nnr_lines:
            print('Dutch defs: ', len(nnr_lines))
            for obj in tqdm(nnr_lines, desc='Saving NNR lines'):
                json.dump(obj, out, ensure_ascii=False)
                out.write('\n')
if ner_lines:
    with open(NER_lines_file, 'w+', encoding='utf-8') as out:
        print('NER defs: ', len(ner_lines))
        for obj in tqdm(ner_lines, desc='Saving NER lines'):
            json.dump(obj, out, ensure_ascii=False)
            out.write('\n')
if other:
    with open(other_file, 'w+', encoding='utf-8') as out:
        print('Other defs: ', len(other))
        for obj in tqdm(other, desc='other lines'):
            json.dump(obj, out, ensure_ascii=False)
            out.write('\n')

629738


Sorting lines: 100%|██████████| 629738/629738 [00:16<00:00, 37447.37it/s] 


Dutch defs:  611444


Saving NNR lines: 100%|██████████| 611444/611444 [00:45<00:00, 13523.78it/s]


In [37]:
print(len(other))

853


In [39]:
for i in other:
    if i.get('lang_code', '') != 'fr':
        display(i)

{'word': 'ja',
 'lang_code': 'af',
 'lang': 'Afrikaans',
 'pos': 'adv',
 'senses': [{'glosses': ['inderdaad']}],
 'categories': ['Bijwoord in het Afrikaans', 'Woorden in het Afrikaans'],
 'antonyms': [{'word': 'nee'}],
 'descendants': [{'lang_code': 'en', 'lang': 'Engels', 'word': 'ja'}]}

{'word': 'ja',
 'lang_code': 'dum',
 'lang': 'Middelnederlands',
 'pos': 'adv',
 'senses': [{'glosses': ['ja']}],
 'categories': ['Bijwoord in het Middelnederlands',
  'Woorden in het Middelnederlands',
  'Woorden in het Middelnederlands met IPA-weergave',
  'Woorden in het Middelnederlands van lengte 2'],
 'etymology_texts': ['Afgeleid van het Oudsaksische ja'],
 'sounds': [[{'ipa': '/jaː/'}]],
 'descendants': [{'lang_code': 'nl',
   'lang': 'Nederlands',
   'word': 'ja',
   'descendants': [{'lang_code': 'af',
     'lang': 'Afrikaans',
     'word': 'ja',
     'descendants': [{'lang_code': 'en', 'lang': 'Engels', 'word': 'ja'},
      {'lang_code': 'pt', 'lang': 'Portugees', 'word': 'iá'}]}]}]}

{'word': 'ja',
 'lang_code': 'unknown',
 'lang': 'Oudnederlands',
 'pos': 'adv',
 'senses': [{'glosses': ['ja; duidt bevestiging, instemming, toestemming, inwilliging of toegeving aan']}],
 'categories': ['Bijwoord in het Oudnederlands',
  'Woorden in het Oudnederlands'],
 'etymology_texts': ['Afgeleid van het Middelnederduitse ja'],
 'descendants': [{'lang_code': 'dum',
   'lang': 'Middelnederlands',
   'word': 'ja',
   'descendants': [{'lang_code': 'nl',
     'lang': 'Nederlands',
     'word': 'ja',
     'descendants': [{'lang_code': 'af',
       'lang': 'Afrikaans',
       'word': 'ja',
       'descendants': [{'lang_code': 'en', 'lang': 'Engels', 'word': 'ja'},
        {'lang_code': 'pt', 'lang': 'Portugees', 'word': 'iá'}]}]}]}]}

{'word': 'een',
 'lang_code': 'dum',
 'lang': 'Middelnederlands',
 'pos': 'article',
 'senses': [{'glosses': ["een, 'n; een onbepaald lidwoord"]}],
 'categories': ['Lidwoord in het Middelnederlands',
  'Woorden in het Middelnederlands',
  'Woorden in het Middelnederlands met IPA-weergave',
  'Woorden in het Middelnederlands van lengte 3'],
 'etymology_texts': ['Afgeleid van het Oudnederlandse ēn / ein'],
 'sounds': [[{'ipa': '/eɛ̯n/'}]],
 'descendants': [{'lang_code': 'nl', 'lang': 'Nederlands', 'word': 'een'},
  {'lang_code': 'nl', 'lang': 'Nederlands', 'word': "'n"}]}

{'word': 'een',
 'lang_code': 'dum',
 'lang': 'Middelnederlands',
 'pos': 'num',
 'senses': [{'glosses': ['een, één; het kleinste hele getal, in Arabische cijfers 1, in Romeinse cijfers I']}],
 'categories': ['Hoofdtelwoord in het Middelnederlands',
  'Woorden in het Middelnederlands',
  'Woorden in het Middelnederlands met IPA-weergave',
  'Woorden in het Middelnederlands van lengte 3'],
 'etymology_texts': ['Afgeleid van het Oudnederlandse ēn / ein'],
 'sounds': [[{'ipa': '/eɛ̯n/'}]],
 'descendants': [{'lang_code': 'nl',
   'lang': 'Nederlands',
   'word': 'een',
   'descendants': [{'lang_code': 'af', 'lang': 'Afrikaans', 'word': 'een'}]}]}

...

{'word': 'Contacum',
 'lang_code': 'la',
 'lang': 'Latijn',
 'pos': 'name',
 'senses': [{'glosses': ['een stad in de Belgische provincie Antwerpen.'],
   'tags': ['toponymic'],
   'categories': ['Aardrijkskunde van België_in_het_Latijn']}],
 'categories': ['Eigennaam in het Latijn',
  'Woorden in het Latijn',
  'Woorden in het Latijn met IPA-weergave',
  'Woorden in het Latijn met audioweergave'],
 'sounds': [[{'ipa': '[kɔntɑkum]'}]],
 'synonyms': [{'word': 'Condacum'}, {'word': 'Contiacum'}],
 'translations': [{'lang_code': 'lt',
   'lang': 'Litouws',
   'word': 'Kontichas',
   'sense': 'een stad in de Belgische provincie Antwerpen.',
   'sense_index': 1},
  {'lang_code': 'nl',
   'lang': 'Nederlands',
   'word': 'Kontich',
   'sense': 'een stad in de Belgische provincie Antwerpen.',
   'sense_index': 1}]}

{'word': 'Contiacum',
 'lang_code': 'la',
 'lang': 'Latijn',
 'pos': 'name',
 'senses': [{'glosses': ['een stad in de Belgische provincie Antwerpen.'],
   'tags': ['toponymic'],
   'categories': ['Aardrijkskunde van België_in_het_Latijn']}],
 'categories': ['Eigennaam in het Latijn',
  'Woorden in het Latijn',
  'Woorden in het Latijn met IPA-weergave',
  'Woorden in het Latijn met audioweergave'],
 'sounds': [[{'ipa': '[kɔntsiɑkum]'}]],
 'synonyms': [{'word': 'Condacum'}, {'word': 'Contacum'}],
 'translations': [{'lang_code': 'lt',
   'lang': 'Litouws',
   'word': 'Kontichas',
   'sense': 'een stad in de Belgische provincie Antwerpen.',
   'sense_index': 1},
  {'lang_code': 'nl',
   'lang': 'Nederlands',
   'word': 'Kontich',
   'sense': 'een stad in de Belgische provincie Antwerpen.',
   'sense_index': 1}]}

{'word': 'Feliciacum',
 'lang_code': 'la',
 'lang': 'Latijn',
 'pos': 'name',
 'senses': [{'glosses': ['dorp in de Belgische provincie Oost-Vlaanderen.'],
   'tags': ['toponymic'],
   'categories': ['Aardrijkskunde van België_in_het_Latijn']}],
 'categories': ['Eigennaam in het Latijn',
  'Woorden in het Latijn',
  'Woorden in het Latijn met IPA-weergave',
  'Woorden in het Latijn met audioweergave'],
 'sounds': [[{'ipa': '[fɛlitʃiɑkum]'}]],
 'synonyms': [{'word': 'Felsecum'}],
 'translations': [{'lang_code': 'nl',
   'lang': 'Nederlands',
   'word': 'Velzeke',
   'sense': 'dorp in de Belgische provincie Oost-Vlaanderen',
   'sense_index': 1}]}

{'word': 'Soniacum',
 'lang_code': 'la',
 'lang': 'Latijn',
 'pos': 'name',
 'senses': [{'glosses': ['een stad in de Belgische provincie Henegouwen.'],
   'tags': ['toponymic'],
   'categories': ['Aardrijkskunde van België_in_het_Latijn']}],
 'categories': ['Eigennaam in het Latijn',
  'Woorden in het Latijn',
  'Woorden in het Latijn met IPA-weergave',
  'Woorden in het Latijn met audioweergave'],
 'sounds': [[{'ipa': '[sɔniɑkum]'}]],
 'synonyms': [{'word': 'Sunniacum'}],
 'translations': [{'lang_code': 'fr',
   'lang': 'Frans',
   'word': 'Soignies',
   'sense': 'een stad in de Belgische provincie Henegouwen',
   'sense_index': 1},
  {'lang_code': 'li',
   'lang': 'Limburgs',
   'word': 'Zinnik',
   'sense': 'een stad in de Belgische provincie Henegouwen',
   'sense_index': 1},
  {'lang_code': 'nl',
   'lang': 'Nederlands',
   'word': 'Zinnik',
   'sense': 'een stad in de Belgische provincie Henegouwen',
   'sense_index': 1},
  {'lang_code': 'pcd',
   'lang': 'Picardisch',
   'word':

{'word': 'Turnacum',
 'lang_code': 'la',
 'lang': 'Latijn',
 'pos': 'name',
 'senses': [{'glosses': ['Doornik, een stad in de Belgische provincie Henegouwen.'],
   'tags': ['toponymic'],
   'categories': ['Aardrijkskunde van België_in_het_Latijn']}],
 'categories': ['Eigennaam in het Latijn',
  'Woorden in het Latijn',
  'Woorden in het Latijn met IPA-weergave',
  'Woorden in het Latijn met audioweergave'],
 'sounds': [[{'ipa': '[tuɾnɑkum]'}]],
 'synonyms': [{'word': 'Tornacum Nerviorum'}],
 'translations': [{'lang_code': 'fr',
   'lang': 'Frans',
   'word': 'Tournai',
   'sense': 'een stad in de Belgische provincie Henegouwen',
   'sense_index': 1},
  {'lang_code': 'nl',
   'lang': 'Nederlands',
   'word': 'Doornik',
   'sense': 'een stad in de Belgische provincie Henegouwen',
   'sense_index': 1}]}

In [24]:
if ner_lines:
    print('English defs: ', len(ner_lines))
    with open(NER_lines_file, 'w+',encoding='utf-8') as out:
        for obj in tqdm(ner_lines, desc='Saving NER lines'):
            json.dump(obj, out, ensure_ascii=False)
            out.write('\n')
if other:
    print('Other lines:', len(other))
    display(other)


English defs:  17441


Saving NER lines: 100%|██████████| 17441/17441 [00:00<00:00, 25153.37it/s]

Other lines: 20


[{'word': 'Aldenarda',
  'lang_code': 'la',
  'lang': 'Latijn',
  'pos': 'name',
  'pos_title': 'Eigennaam',
  'senses': [{'glosses': ['een stad in de Belgische provincie Oost-Vlaanderen'],
    'tags': ['toponymic'],
    'categories': ['Aardrijkskunde van België_in_het_Latijn']}],
  'categories': ['Eigennaam in het Latijn',
   'Woorden in het Latijn',
   'Woorden in het Latijn met IPA-weergave',
   'Woorden in het Latijn met audioweergave'],
  'sounds': [{'audio': 'la-Aldenarda.ogg',
    'ogg_url': 'https://commons.wikimedia.org/wiki/Special:FilePath/la-Aldenarda.ogg',
    'mp3_url': 'https://upload.wikimedia.org/wikipedia/commons/transcoded/2/2e/La-Aldenarda.ogg/La-Aldenarda.ogg.mp3'},
   {'ipa': '[ɑldɛnɑɾdɑ]'}],
  'translations': [{'lang_code': 'nl',
    'lang': 'Nederlands',
    'word': 'Oudenaarde',
    'sense': 'een stad in de Belgische provincie Oost-Vlaanderen',
    'sense_index': 1}]},
 {'word': 'Alostum',
  'lang_code': 'la',
  'lang': 'Latijn',
  'pos': 'name',
  'pos_title':

## Process Raw EN File

In [65]:
total_en_lines = count_lines_with_progress(ERAW_FILE)
print(total_en_lines)

Counting Lines: 100%|██████████| 21.3G/21.3G [00:17<00:00, 1.23GB/s]

10329308


In [309]:
previous_save_path = Path(WIKT_PREPROCESSING_DIR, '07-11-25')

In [ ]:
current_save_path = Path(WIKT_PREPROCESSING_DIR, '09-11-25')

EER_save_path = Path(current_save_path, 'EER')
ENR_save_path = Path(current_save_path, 'ENR')

Path.mkdir(current_save_path, exist_ok=True)
Path.mkdir(EER_save_path, exist_ok=True)
Path.mkdir(ENR_save_path, exist_ok=True)

### Utilities

In [ ]:
def keep_obj(obj: dict) -> bool: 
    if "lang_code" not in obj:
        return False
    if obj["lang_code"] not in ["nl", "en", "simple"]:
        return False
    if not check_has_valid_chars(obj['word']):
        return False
    return True


def keep_obj_before_load(obj_str: str) -> bool:
    if has_cjk_or_arabic_fast(obj_str):
        return False
    return True
        

### Filter json translations and translation related categories 

In [ ]:
def filter_dutch_translations_regex(obj_str: str):
    translations_pattern = r'"translations"\s*:\s*\[({.*?})\]'
    dutch_translation_pattern = r'\{[^{}]*?"lang"\s*:\s*"Dutch"[^{}]*?\}'
    translations_block = re.compile(translations_pattern, re.DOTALL)
    dutch_object = re.compile(dutch_translation_pattern, re.DOTALL)
    match = translations_block.search(obj_str)
    
    while match is not None:
        start, end = match.span()
        m = match.group(0)
        dn = dutch_object.findall(m)
        if dn:
            dn = [safe_dict(x) for x in dn]
            str_dn = '[' + ', '.join(json.dumps(x) for x in dn) + ']'
            obj_str = obj_str[:start] + '"translations": ' + str_dn + obj_str[end:]
            match = translations_block.search(obj_str, start + len('"translations": ' + str(dn)))
        else:
            
            to_remove_end = end
            if end < len(obj_str) and obj_str[end] == ',':
                to_remove_end += 1
            obj_str = obj_str[:start] + obj_str[to_remove_end:]
            match = translations_block.search(obj_str, start)

    return obj_str

In [11]:
def filter_engels_translations(obj_str: str):
    translations_pattern = r'"translations"\s*:\s*\[({.*?})\]'
    engels_translation_pattern = r'\{[^{}]*?"lang_code"\s*:\s*"en"[^{}]*?\}'
    translations_block = re.compile(translations_pattern, re.DOTALL)
    engels_object = re.compile(engels_translation_pattern, re.DOTALL)
    match = translations_block.search(obj_str)
    
    while match is not None:
        start, end = match.span()
        m = match.group(0)
        dn = engels_object.findall(m)
        if dn:
            dn = [safe_dict(x) for x in dn]
            str_dn = '[' + ', '.join(json.dumps(x) for x in dn) + ']'
            obj_str = obj_str[:start] + '"translations": ' + str_dn + obj_str[end:]
            match = translations_block.search(obj_str, start + len('"translations": ' + str(dn)))
        else:
            
            to_remove_end = end
            if end < len(obj_str) and obj_str[end] == ',':
                to_remove_end += 1
            obj_str = obj_str[:start] + obj_str[to_remove_end:]
            match = translations_block.search(obj_str, start)


In [8]:
def filter_line(obj_str: str) -> str:
    if '"lang": "Nederlands"' in obj_str[0:100] or '"lang": "Engels"' in obj_str[0:100]:
        obj_str = filter_engels_translations(obj_str)
    else:
        obj_str = filter_translations_regex(obj_str)
    return obj_str

In [ ]:
def filter_obj(obj: dict):
    lang_codes_to_keep = ['nl', 'en', 'simple', 'ang', 'dum', 'nds', 'nds-nl', 'odt', 'nds-nl', 'enm']
    
    if 'anagrams' in obj:
        obj.pop('anagrams')
    if 'proverbs' in obj:
        obj.pop('proverbs')
    if 'pos_title' in obj:
        obj.pop('pos_title')
    
    if 'sounds' in obj:
        sounds = obj['sounds']
        remove_sounds = ['audio','ogg_url','mp3_url']
        new_sounds = []
        for i in sounds:
            ns = []
            for k in i.keys():
                if k not in remove_sounds:
                    ns.append(i)
            if ns:
                new_sounds.append(ns)
            
        obj['sounds'] = new_sounds
    if 'descendants' in obj:
        descendants = obj['descendants']
        desc = [d for d in descendants if d.get("lang_code") in lang_codes_to_keep or  d.get("lang_code", '') == '']
        obj['descendants'] = desc
    if 'derived' in obj:
        derived = obj['derived']
        desc = [d for d in derived if d.get("lang_code") in lang_codes_to_keep or d.get("lang_code", '') == '']
        obj['derived'] = desc
    if 'senses' in obj:
        new_senses = []
        for sense in obj['senses']:
            if 'wikidata' in sense:
                sense.pop('wikidata')
            if 'attestations' in sense:
                sense.pop('attestations')
            if 'wikipedia' in sense:
                sense.pop('wikipedia')
            if 'senseid' in sense:
                sense.pop('senseid')
            if 'translations' in sense:
                sense_translations = [s for s in sense['translations'] if s.get('lang_code') in lang_codes_to_keep]
                sense['translations'] = sense_translations
            if 'translation' in sense and 'translations' not in sense:
                sense_translations = [s for s in sense['translation'] if s.get('lang_code') in lang_codes_to_keep]
                sense['translation'] = sense_translations
            new_senses.append(sense)
        obj['senses'] = new_senses
    if "categories" in obj:
        categories = obj["categories"]
        new_categories = []
        if isinstance(categories, list):
            for i, cat in enumerate(categories):
                if isinstance(cat, str):
                    if cat not in ["Terms with Dutch translations", "Terms with English translations"]:
                        if cat.startswith('Terms with') and cat.endswith('translations'):
                            categories.remove(cat)
                        elif cat.startswith('Pages with') and (cat.endswith('entry') or cat.endswith('entries')):
                            categories.remove(cat)
                        else:
                            new_categories.append(cat)
        obj["categories"] = new_categories
    if 'translations' in obj:
        translations = obj['translations']
        translations = [s for s in translations if s.get('lang_code') in lang_codes_to_keep]
    return obj


In [32]:
current_save_path = Path(WIKT_PREPROCESSING_DIR, 'en')

In [72]:

batch_size = 100000
error_lines = []
from pprint import pp, pprint
enl_lines = []
other_lines = []

with open(EER_file2, "w+", encoding='utf-8') as output_file:
    with open(ERAW_FILE, "r", encoding='utf-8') as f:
        batch_list = []
        for i, line in tqdm(enumerate(f), total=total_en_lines):
            
            try:
                if not keep_obj_before_load(line):
                    continue

                try:
                    obj = json.loads(line)
                
                    obj = filter_nl_obj(obj)
                    # obj = filter_obj(obj)
                    if obj:
                        if obj['lang_code'] == 'en':
                            batch_list.append(obj)

                            if i % batch_size == 0 and i > 0:
                                print(f"Writing batch at line {i}")
                                for obj in batch_list:
                                    json.dump(obj, output_file, ensure_ascii=False)
                                    output_file.write("\n")
                                batch_list = []
                        elif obj['lang_code'] == 'nl':
                            enl_lines.append(obj)
                        else:
                            other_lines.append(obj)
                except Exception as e1:
                    error_lines.append((i, line))
                    print(f"Error parsing JSON on line {i} ", e1)
            except Exception as e:
                print(i, e)
                raise e
        for obj in batch_list:
            json.dump(obj, output_file, ensure_ascii=False)
            output_file.write("\n")  

  2%|▏         | 199631/10329308 [00:19<15:09, 11141.95it/s]

Writing batch at line 200000


  5%|▍         | 499715/10329308 [01:02<09:45, 16791.62it/s] 

Writing batch at line 500000


  8%|▊         | 799721/10329308 [01:32<08:30, 18677.14it/s] 

...

 95%|█████████▍| 9799335/10329308 [10:01<00:20, 26034.99it/s]

Writing batch at line 9800000


 98%|█████████▊| 10098765/10329308 [10:22<00:56, 4068.27it/s] 

Writing batch at line 10100000


100%|██████████| 10329308/10329308 [10:36<00:00, 16216.05it/s]


In [73]:
with open(ENR_file2, "w+", encoding='utf-8') as output_file:
    for obj in enl_lines:
        json.dump(obj, output_file, ensure_ascii=False)
        output_file.write("\n")

In [75]:
print(len(enl_lines))

140758


In [74]:
with open(other_file, 'a+', encoding='utf-8', errors='ignore') as out:
    for obj in other_lines:
        json.dump(obj, out, ensure_ascii=False)
        out.write('\n')

In [76]:
print(len(other_lines))

109693


In [ ]:
temp_output_file = Path(current_save_path, "ERAW_filtered.jsonl")
batch_size = 100000
error_lines = []
from pprint import pp, pprint
enl_lines = []

with open(temp_output_file, "w+", encoding='utf-8') as output_file:
    with open(ERAW_FILE, "r", encoding='utf-8') as f:
        batch_list = []
        for i, line in tqdm(enumerate(f), total=total_en_lines):
            
            try:
                if not keep_obj_before_load(line):
                    continue
               
                line = filter_line(line)
                try:
                    obj = json.loads(line)
                
                    if not keep_obj(obj):
                        continue
                    
                    obj = filter_obj(obj)
                    batch_list.append(obj)

                    if i % batch_size == 0 and i > 0:
                        print(f"Writing batch at line {i}")
                        for obj in batch_list:
                            json.dump(obj, output_file, ensure_ascii=False)
                            output_file.write("\n")
                        batch_list = []
                      
                except Exception as e1:
                    error_lines.append((i, line))
                    print(f"Error parsing JSON on line {i} ", e1)
            except Exception as e:

                print(f.tell())
                raise e
        for obj in batch_list:
            json.dump(obj, output_file, ensure_ascii=False)
            output_file.write("\n")  

### Split Json

In [37]:
ERAW_filtered_file = Path(current_save_path, "ERAW_filtered.jsonl")
count_lines_with_progress(ERAW_filtered_file)

Counting Lines: 100%|██████████| 1.75G/1.75G [00:01<00:00, 1.48GB/s]


1558141

In [40]:
EER_file = Path(EER_DIR, 'EER.jsonl')
ENR_file = Path(ENR_DIR, 'ENR.jsonl')

In [41]:
EER_batch = []
ENR_batch = []
batch_size = 10000
with open(ERAW_filtered_file, 'r', encoding='utf-8') as f:
    with open(EER_file, 'w+', encoding='utf-8') as out:
        for line in tqdm(f, total=count_lines_with_progress(ERAW_filtered_file)):
        
            obj = json.loads(line)
            if obj["lang_code"] == "en":
                EER_batch.append(obj)
                if len(EER_batch) >= batch_size:
                    for item in EER_batch:
                        json.dump(item, out, ensure_ascii=False)
                        out.write("\n")
                    EER_batch = []
            elif obj["lang_code"] == "nl":
                ENR_batch.append(obj)
        if EER_batch:
            for oer in EER_batch:
                json.dump(oer, out, ensure_ascii=False)
                out.write("\n")
if ENR_batch:
    with open(ENR_file, 'w+', encoding='utf-8') as enr_f:
        for obj in ENR_batch:
            json.dump(obj, enr_f, ensure_ascii=False)
            enr_f.write("\n")

Counting Lines: 100%|██████████| 1.75G/1.75G [00:01<00:00, 1.35GB/s]
100%|██████████| 1558141/1558141 [02:24<00:00, 10800.35it/s]


In [42]:
print(count_lines_with_progress(EER_file))
print(count_lines_with_progress(ENR_file))

Counting Lines: 100%|██████████| 1.57G/1.57G [00:01<00:00, 1.40GB/s]


1417383


Counting Lines: 100%|██████████| 175M/175M [00:00<00:00, 1.42GB/s]

140758


In [321]:
print(count_lines_with_progress(EER_file))
print(count_lines_with_progress(ENR_file))

Counting Lines: 100%|██████████| 1.63G/1.63G [00:01<00:00, 1.52GB/s]


1423864


Counting Lines: 100%|██████████| 204M/204M [00:00<00:00, 1.53GB/s]

140922


0.0014376743618479094

In [ ]:
# language	gloss definitions	change	entries	change	gloss entries	change	form definitions	change	total definitions	change
# Total	        4964041		            9977204		    4392194		                6286693		                11250734	

# Dutch	        79363	        +350	141111	+503	66094	        +362	    78942	        +207	    158305	           +557
# English	    949129	        +3110	1425914	+6586	901398	        +4405	    713614	        +4162	    1662743	            +7272

Counting Lines: 100%|██████████| 204M/204M [00:00<00:00, 1.12GB/s]


140922

### Get Keys

In [67]:
def get_subkeys(line):
    structure_dict = {}
    line_type = type(line)
    if not line:
        return line_type
    elif isinstance(line, str):
        return str
    elif isinstance(line, int):
        return int
    elif isinstance(line, dict):
        for k, v in line.items():
            structure_dict[k] = get_subkeys(v)
    elif isinstance(line, list):
        subkeys_list = []
        substruct_count = []
        for i, v in enumerate(line):
            substruct = (dict, get_subkeys(v))
            if substruct not in subkeys_list:
                subkeys_list.append(substruct)
                substruct_count.append(1)
            else:
                index = subkeys_list.index(substruct)
                substruct_count[index] += 1
        return (f'unique:{len(subkeys_list)}', subkeys_list)
    else:
        print(line)
    return structure_dict

In [333]:
def get_subkeysV2(line):
    structure_dict = {}
    line_type = type(line)
    if not line:
        return line_type
    elif isinstance(line, str):
        return str
    elif isinstance(line, int):
        return int
    elif isinstance(line, dict):
        for k, v in line.items():
            structure_dict[k] = get_subkeysV2(v)
    elif isinstance(line, list):
        subkeys_list = []
        substruct_count = []
        line_tuple_list = []
        keys_set = set()
        for i, v in enumerate(line):
            substruct = get_subkeysV2(v)
            if isinstance(substruct, dict):
                for k in substruct.keys():
                    keys_set.update(k)
            if substruct not in subkeys_list:
                subkeys_list.append(substruct)
                
                substruct_count.append(1)
            else:
                index = subkeys_list.index(substruct)
                substruct_count[index] += 1
        return (keys_set, subkeys_list)
    else:
        print(line)
    return structure_dict

In [ ]:
def print_subkeys(structure, depth=0):
    indent = '  ' * depth
    if isinstance(structure, dict):
        for key, value in structure.items():
            print(f"{indent}{key}:")
            print_subkeys(value, depth + 1)
    elif isinstance(structure, list):
        for i, item in enumerate(structure):
            print(f"{indent}- Item {i}:")
            print_subkeys(item, depth + 1)
    else:
        print(f"{indent}{structure}")

In [ ]:
def compare_structures(template_dict, new_dict):
    template_keys = template_dict.keys()
    template_values = template_dict.values()
    if template_dict == new_dict:
        return template_dict
    
    for k, v in new_dict:
        if k not in template_dict:
            template_dict[k] = new_dict[k]
        else:
            if isinstance(v, list):
                pass
    
    # return structure_dict

In [ ]:
import typing
from collections import Counter
structures = []
bad_lines = []
counts_index = 3

with open(ENR_file, 'r', encoding='utf-8',errors='ignore') as f:
    lines = f.readlines()
    template_structure = json.loads(lines[0])
    for line in tqdm(lines, total=len(lines)):
        try:
            loaded = json.loads(line)
            keys = loaded.keys()

            lines.append(loaded)

            d = get_subkeys(loaded)
            
            structures.append(d)
        except Exception as e:
            bad_lines.append(line)
        

281844it [00:22, 12698.46it/s]                            


In [332]:
display(structures[0:10])

[{'senses': ('unique:1',
   [(dict,
     {'links': ('unique:1', [(dict, ('unique:1', [(dict, str)]))]),
      'synonyms': ('unique:1', [(dict, {'word': str})]),
      'glosses': ('unique:1', [(dict, str)]),
      'tags': ('unique:1', [(dict, str)]),
      'categories': ('unique:1', [(dict, str)]),
      'wikipedia': ('unique:1', [(dict, str)])})]),
  'pos': str,
  'head_templates': ('unique:1',
   [(dict,
     {'name': str,
      'args': {'1': str, '2': str, '3': str},
      'expansion': str})]),
  'forms': ('unique:1',
   [(dict, {'form': str, 'tags': ('unique:1', [(dict, str)])})]),
  'derived': ('unique:1', [(dict, {'word': str})]),
  'descendants': ('unique:2',
   [(dict, {'lang': str, 'lang_code': str, 'word': str}),
    (dict,
     {'lang': str,
      'lang_code': str,
      'word': str,
      'tags': ('unique:1', [(dict, str)])})]),
  'sounds': ('unique:2',
   [(dict, {'ipa': str}),
    (dict, {'audio': str, 'ogg_url': str, 'mp3_url': str})]),
  'hyphenations': ('unique:1',
   [

### Get definitions and translations